In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2 as cv 
import glob
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler,Subset
from torchvision import transforms
from IPython.display import HTML
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torchvision  import datasets 
from sklearn.model_selection import KFold
import optuna
from optuna.trial import Trial
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Subset, DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
combined_data = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

In [4]:
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert to Tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])

In [5]:
combinedData = datasets.ImageFolder(combined_data, transform=transform)

In [6]:
combinedData

Dataset ImageFolder
    Number of datapoints: 10905
    Root location: /kaggle/input/deepfake-and-real-images/Dataset/Test
    StandardTransform
Transform: Compose(
               Resize(size=(299, 299), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [7]:
kf = KFold(n_splits=3, shuffle=True, random_state=42) 

In [8]:
samples = len(combinedData)
samples

10905

In [9]:
folds = list(kf.split(np.arange(samples)))

In [10]:
folds

[(array([    1,     2,     4, ..., 10900, 10901, 10902]),
  array([    0,     3,     8, ..., 10899, 10903, 10904])),
 (array([    0,     1,     3, ..., 10902, 10903, 10904]),
  array([    2,    15,    18, ..., 10889, 10900, 10901])),
 (array([    0,     2,     3, ..., 10901, 10903, 10904]),
  array([    1,     4,     5, ..., 10896, 10897, 10902]))]

# Split our data into 3 Kfolds

In [11]:
for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
    print(f"Fold {fold_idx + 1}")
    
    train_size = int(0.75* len(train_val_idx)) 
    val_size = len(train_val_idx) - train_size #25
    
    train_idx = train_val_idx[:train_size]
    val_idx = train_val_idx[train_size:]
    
    # Create subsets
    train_set = Subset(combinedData, train_idx)
    val_set = Subset(combinedData, val_idx)
    test_set = Subset(combinedData, test_idx)
    
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
    
    print(f"Train size: {len(train_set)}")
    print(f"Validation size: {len(val_set)}")
    print(f"Test size: {len(test_set)}")

Fold 1
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 2
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 3
Train size: 5452
Validation size: 1818
Test size: 3635


# Define our model inception

In [12]:
def get_model(name): 
    model = models.inception_v3(pretrained=True)
    # Freeze all parameters initially
    for param in model.parameters():
        param.requires_grad = False
    
    # Modify the fully connected layer for binary classification
    model.fc = nn.Linear(model.fc.in_features, 2)
    return model

In [13]:
# fine tune
def get_model(name="inception_v3"):
    # Load the pre-trained InceptionV3 model
    model = models.inception_v3(pretrained=True)

    # Freeze all parameters initially
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the last two convolutional blocks
    for layer_name in ['Mixed_6e', 'Mixed_7a', 'Mixed_7b', 'Mixed_7c']:  # These are the last two blocks in InceptionV3
        layer = getattr(model, layer_name)
        for param in layer.parameters():
            param.requires_grad = True

    # Replace the fully connected layer to match the output classes
    # Assuming you have 2 classes for binary classification
    model.fc = nn.Linear(model.fc.in_features, 2)

    # Print trainable parameters count
    def count_trainable_params(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Total trainable parameters: {count_trainable_params(model):,}")

    return model

# Training function

In [13]:
def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Handle InceptionOutputs if present
        if isinstance(outputs, tuple):
            main_output, aux_output = outputs
            loss = criterion(main_output, labels) + 0.4 * criterion(aux_output, labels)  # Auxiliary loss with a weight of 0.4
        else:
            loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update running loss and accuracy
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs[0] if isinstance(outputs, tuple) else outputs, 1)  # Use main output for predictions
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    # Calculate epoch loss and accuracy
    epoch_loss = running_loss / total
    epoch_acc = correct / total

    return epoch_loss, epoch_acc


# fine tune training

In [14]:
#Fine-tune

def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Add gradient clipping to prevent exploding gradients
    max_grad_norm = 1.0
    
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        if isinstance(outputs, tuple):  # Handle InceptionV3 with auxiliary outputs
            main_output, aux_output = outputs
            loss = criterion(main_output, labels) + 0.4 * criterion(aux_output, labels)
        else:
            main_output = outputs
            loss = criterion(main_output, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()

        # Update metrics
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(main_output, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


# Eval function

In [14]:
def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Optimize the Learning rate using Optuna

In [16]:
def LR_optimization(trial: Trial):
    # Suggest a learning rate
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    
    # Store results for each fold
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"\nFold {fold_idx + 1}")
        
        # Split train and validation sets
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        for model_name in ["inception_v3"]:
            print(f"\nTraining model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            # Initialize the Inception model
            model = get_model(model_name).to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                # Train and evaluate
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                val_loss, val_acc = evaluate(model, val_loader, criterion)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            # Evaluate on validation set after training
            _, val_acc = evaluate(model, val_loader, criterion)
            fold_results.append(val_acc)

    # Return the mean validation accuracy
    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc


In [16]:
#Fine-tune



def LR_optimization(trial: Trial):
    # Use a smaller learning rate range for fine-tuning
    lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
    
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"\nFold {fold_idx + 1}")
        
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        for model_name in ["inception_v3"]:
            print(f"\nFine-tuning model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            model = get_model(model_name).to(device)  # Ensure get_model returns InceptionV3
            criterion = nn.CrossEntropyLoss()
            
            # Use a smaller weight decay for fine-tuning
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            
            # Add learning rate scheduler for fine-tuning
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='max', factor=0.5, patience=2, verbose=True
            )

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                val_loss, val_acc = evaluate(model, val_loader, criterion)
                
                # Update learning rate based on validation accuracy
                scheduler.step(val_acc)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            _, val_acc = evaluate(model, val_loader, criterion)
            fold_results.append(val_acc)

    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc


In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(LR_optimization, n_trials=5)

best_trial = study.best_trial
print("\nBest Trial:")
print(f"  Value (Mean Validation Accuracy): {best_trial.value}")
print(f"  Params: {best_trial.params}")

[I 2024-12-23 18:49:30,352] A new study created in memory with name: no-name-bc09e58e-625e-4055-b361-cab1d7dad806
<ipython-input-16-a86a1a29e4a9>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-


Fold 1

Fine-tuning model: INCEPTION_V3 on Fold 1


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 209MB/s] 


Total trainable parameters: 11,125,506


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Train Loss: 3.5821, Train Acc: 0.9076
Val Loss: 0.7252, Val Acc: 0.6342

Epoch 4/5 | Learning Rate: 0.000023


Train Loss: 3.4809, Train Acc: 0.9540
Val Loss: 0.7963, Val Acc: 0.6419

Epoch 5/5 | Learning Rate: 0.000023


Train Loss: 3.4217, Train Acc: 0.9807
Val Loss: 1.0742, Val Acc: 0.5968


[I 2024-12-23 20:16:03,234] Trial 4 finished with value: 0.6017601760176018 and parameters: {'lr': 2.3036409951563454e-05}. Best is trial 2 with value: 0.7427576090942427.



Best Trial:
  Value (Mean Validation Accuracy): 0.7427576090942427
  Params: {'lr': 0.0003267594210449535}


# After the Optuna optimization process, we will use the best trial to set the learning rate

In [15]:
# Best learning rate from Optuna

fine_best_lr = 0.0003267594210449535
best_lr = 0.00015312307074091057

In [16]:
final_results = {}

for model_name in ["inception_v3"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    # Initialize model
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    
    # Create DataLoaders (assuming combinedData is split into train and test datasets)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    # Train the model
    for epoch in range(10):  # Example: 10 epochs
        print(f"\nEpoch {epoch + 1}/10")
        
        # Train and evaluate
        train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    
    # Final evaluation on the test set
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    
    # Store results
    final_results[model_name] = {
        "Test Loss": test_loss,
        "Test Accuracy": test_acc
    }

# Print final results
print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"{model_name.upper()}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


Training model: INCEPTION_V3 with best learning rate 0.00015312307074091057


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 198MB/s] 



Epoch 1/10


Train Loss: 3.9750, Train Acc: 0.6759

Epoch 2/10


Train Loss: 3.9401, Train Acc: 0.6935

Epoch 3/10


...

Train Loss: 3.8860, Train Acc: 0.7254

Epoch 8/10


Train Loss: 3.8803, Train Acc: 0.7322

Epoch 9/10


Train Loss: 3.8738, Train Acc: 0.7287

Epoch 10/10


Train Loss: 3.8782, Train Acc: 0.7276


Test Loss: 0.6462, Test Acc: 0.5989

Final Results:
INCEPTION_V3:
  Test Loss: 0.6462
  Test Accuracy: 0.5989


In [17]:
# Function to evaluate the model and get performance metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Initialize cross-validation metrics storage
confusion_matrices = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

best_lr = 0.00015312307074091057

for model_name in ["inception_v3"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"Fold {fold_idx + 1}")

        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
        
        
        model = get_model(model_name).to(device)
        optimizer = optim.Adam(model.parameters(), lr=best_lr)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        for epoch in range(10):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                if isinstance(outputs , tuple):
                    outputs = outputs.logits

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/10, Loss: {running_loss / len(train_loader):.4f}")

        
        preds, labels = evaluate_model(model, test_loader)

        
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average='weighted', zero_division=0)
        recall = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        cm = confusion_matrix(labels, preds)

        # Store metrics for the fold
        confusion_matrices.append(cm)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold_idx + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{cm}\n")

    # Average metrics across folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)

    # Print final average metrics for the model
    print(f"\nFinal Results for {model_name.upper()}:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1_score:.4f}")
    print(f"Average Confusion Matrix:\n{avg_confusion_matrix}")


Training model: INCEPTION_V3 with best learning rate 0.00015312307074091057
Fold 1
Epoch 1/10, Loss: 0.6287
Epoch 2/10, Loss: 0.5969
Epoch 3/10, Loss: 0.5787
Epoch 4/10, Loss: 0.5698
Epoch 5/10, Loss: 0.5630
Epoch 6/10, Loss: 0.5573
Epoch 7/10, Loss: 0.5431
Epoch 8/10, Loss: 0.5406
Epoch 9/10, Loss: 0.5370
Epoch 10/10, Loss: 0.5358
Fold 1 - Accuracy: 0.6322, Precision: 0.6895, Recall: 0.6322, F1-Score: 0.5965
Confusion Matrix:
[[1711  153]
 [1184  587]]

Fold 2


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.6365
Epoch 2/10, Loss: 0.6028
Epoch 3/10, Loss: 0.5835
Epoch 4/10, Loss: 0.5708
Epoch 5/10, Loss: 0.5639
Epoch 6/10, Loss: 0.5542
Epoch 7/10, Loss: 0.5533
Epoch 8/10, Loss: 0.5454
Epoch 9/10, Loss: 0.5401
Epoch 10/10, Loss: 0.5353
Fold 2 - Accuracy: 0.6757, Precision: 0.7100, Recall: 0.6757, F1-Score: 0.6607
Confusion Matrix:
[[1617  217]
 [ 962  839]]

Fold 3


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.6228
Epoch 2/10, Loss: 0.5902
Epoch 3/10, Loss: 0.5725
Epoch 4/10, Loss: 0.5677
Epoch 5/10, Loss: 0.5486
Epoch 6/10, Loss: 0.5444
Epoch 7/10, Loss: 0.5429
Epoch 8/10, Loss: 0.5404
Epoch 9/10, Loss: 0.5333
Epoch 10/10, Loss: 0.5275
Fold 3 - Accuracy: 0.6283, Precision: 0.6885, Recall: 0.6283, F1-Score: 0.5988
Confusion Matrix:
[[1624  170]
 [1181  660]]


Final Results for INCEPTION_V3:
Average Accuracy: 0.6454
Average Precision: 0.6960
Average Recall: 0.6454
Average F1-Score: 0.6187
Average Confusion Matrix:
[[1650.66666667  180.        ]
 [1109.          695.33333333]]


# Average accuracy of inception_v3 without fine tuning it = 64%
# Average accuracy of inception_v3 with fine tune = 90%
